In [1]:
import requests
class Data():
    def __init__(self):
        #key1 "ad9765417caa9e2fd304e0aea0e2d24f"
        #key2 "4ccf98ac9854f73e845a16a9e300daeb"
        self.headers = {
            "content-type": "application/json",
            "X-RosetteAPI-Key": "4ccf98ac9854f73e845a16a9e300daeb",
            "X-RapidAPI-Key":
            "bc44666f27msh0cea06e275c5146p185e39jsnb306b7888971",
            "X-RapidAPI-Host": "rosetteapi-rosette-v1.p.rapidapi.com"
        }
        self.payload = {}

    def get_token(self, text):
        self.payload = {}
        url = "https://rosetteapi-rosette-v1.p.rapidapi.com/tokens"
        self.payload['content'] = text
        response = requests.request("POST",
                                    url,
                                    json=self.payload,
                                    headers=self.headers)
        return eval(response.text)['tokens']

    def get_lang(self, text):
        self.payload = {}
        url = "https://api.rosette.com/rest/v1/language"
        self.payload['content'] = text
        response = requests.request("POST",
                                    url,
                                    json=self.payload,
                                    headers=self.headers)
        return eval(response.text)["languageDetections"][0]['language']

    def get_vec(self, text):
        self.payload = {}
        url = "https://api.rosette.com/rest/v1/semantics/vector"
        self.payload['content'] = text
        self.payload['options'] = {"perToken": True}
        response = requests.request("POST",
                                    url,
                                    json=self.payload,
                                    headers=self.headers)
        return {
            'documentEmbedding': eval(response.text)['documentEmbedding'],
            'tokenEmbeddings': eval(response.text)['tokenEmbeddings']
        }

    def get_posTag(self, text):
        self.payload = {}
        null = ''
        url = "https://rosetteapi-rosette-v1.p.rapidapi.com/morphology/complete"
        self.payload['content'] = text
        response = requests.request("POST",
                                    url,
                                    json=self.payload,
                                    headers=self.headers)

        return {
            "tokens": eval(response.text)["tokens"],
            "posTags": eval(response.text)["posTags"]
        }

    def get_senTag(self, text):
        self.payload = {}
        url = "https://api.rosette.com/rest/v1/sentences"
        self.payload['content'] = text
        response = requests.request("POST",
                                    url,
                                    json=self.payload,
                                    headers=self.headers)
        return eval(response.text)['sentences']
    
    def get_relation(self, text):
        self.payload = {}
        url = "https://api.rosette.com/rest/v1/relationships"
        self.payload['content'] = text
        response = requests.request("POST",
                                    url,
                                    json=self.payload,
                                    headers=self.headers)
        return eval(response.text)['relationships']

    def get_classification(self, text):
        self.payload = {}
        url = "https://api.rosette.com/rest/v1/categories"
        self.payload['content'] = text
        response = requests.request("POST",
                                    url,
                                    json=self.payload,
                                    headers=self.headers)
        return eval(response.text)["categories"]

In [2]:
from wordcloud import WordCloud
import pandas as pd
import altair as alt
from altair_saver import save
import numpy as np
import math
from collections import OrderedDict
from graphviz import Graph

In [3]:
class NLPbrl():
    def __init__(self):
        self.api_data = Data()

    def __word_wash(self, text, stop_word='default'):
        #determine language
        if stop_word == 'default':
            try:
                lang = self.api_data.get_lang(text[:100])
            except:
                print("Error with get language function")
                return False
            if lang == 'zho':
                stop_word = 'pacakge_data/cn_stopwords.txt'
            elif lang == 'eng':
                stop_word = 'pacakge_data/en_stopwords.txt'
            else:
                stop_word = 'pacakge_data/other_stopwords.txt'
        elif stop_word == 'None':
            #check the characters limit
            text_lst = self.__check_limit(text)
            text_token = []
            try:
                for t in text_lst:
                    text_token.extend(self.api_data.get_token(t))
            except:
                print("Error with get token function")
                return False

            washed_token = text_token
            return washed_token

        try:
            stop_file = open(stop_word, 'r', encoding='utf-8')
            stopwords = stop_file.read().split("\n")
            stop_file.close()
        except:
            print('{} is not exist, please check the file!'.format(stop_word))
            return False

        #check the characters limit
        text_lst = self.__check_limit(text)
        text_token = []
        try:
            for t in text_lst:
                text_token.extend(self.api_data.get_token(t))
        except:
            print("Error with get token function")
            return False

        washed_token = []
        for char in text_token:
            if char in stopwords:
                pass
            else:
                washed_token.append(char)
        return washed_token

    def cal_frequency(self, text, stop_word='default'):
        washed_token = self.__word_wash(text, stop_word)
        if washed_token == False:
            return False
        counts = {}
        for word in washed_token:
            counts[word] = counts.get(word, 0) + 1

        counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
        return counts

    def __check_limit(self, text):
        text = text.replace('\n', ' ')
        if len(text.replace(' ', '')) <= 5000:
            return [text]
        else:
            true_text = []
            if '.' in text:
                true_text = self.__check_limit_tool(text, '.')
            elif '。' in text:
                true_text = self.__check_limit_tool(text, '。')
            else:
                true_text = self.__check_limit_tool(text, '')
        return true_text


    def __check_limit_tool(self, text, symbol):
        temp = []
        string = ''
        if symbol != '':
            split_text = text.split(symbol)
        else:
            split_text = text
        for sen in split_text:
            temp_s = string + sen + symbol
            if len(temp_s.replace(' ', '')) > 5000:
                temp.append(string)
                string = sen + symbol
            else:
                string = string + sen + symbol

            if sen == split_text[-1]:
                temp.append(string)
        return temp

    def word_viz(self, text, file_loc, top_num, stop_word='default', cloud_set=WordCloud(font_path='pacakge_data/STKAITI.TTF')):
        washed_token = self.__word_wash(text, stop_word)
        
        if washed_token == False:
            return False
        
        try:
            cloud_set.generate(' '.join(washed_token))
        except:
            print("Error with word cloud.")
            return False
        
        try:
            cloud_set.to_file(file_loc + 'cloud.png')
        except:
            print("Error with saving word cloud.")
            return False

        counts = self.cal_frequency(text, stop_word)
        words = []
        count = []
        for i in counts:
            words.append(i[0])
            count.append(i[1])
        df = pd.DataFrame({'words': words, 'count': count})
        
        if top_num >= len(df):
            top_num = len(df) - 1
            
        chart = (alt.Chart(df[:top_num]).mark_line().encode(
            x='words', y='count').properties(height=400, width=400))
        try:
            save(chart, "chart.html")
        except:
            print("Error with saving chart.")
            return False
        return True

    #text: text which needs to be calculate tfidf
    #document: document_loc to train idf
    #topK, keywords for top x
    def key_extra_tfidf(self, text, document, stop_word='default'):
        if len(text)==0 or len(document)==0 or type(document)!=list:
            print('Error with input type and content.')
            return False
        
        text_wash_lst = []
        for doc in document:
            wash_token = self.__word_wash(doc, stop_word)
            if wash_token == False:
                return False
            text_wash_lst.append(wash_token)
        
        text_wash = self.__word_wash(text, stop_word)
        if text_wash == False:
            return False

        #word dictionary
        set_lst = []
        for i in text_wash_lst:
            set_lst.extend(i)
        wordSet = set(set_lst)

        wordDict_lst = []
        for wash_token in text_wash_lst:
            temp_dic = dict.fromkeys(wordSet, 0)
            for word in wash_token:
                temp_dic[word] += 1
            wordDict_lst.append(temp_dic)
        
        wordDict = dict.fromkeys(wordSet, 0)
        for word in text_wash:
            wordDict[word] += 1
        
        try:
            tfDict = self.__cal_TF(wordDict, text_wash)
            idfDict = self.__cal_IDF(wordDict_lst)

            tfidf = self.__cal_TFIDF(tfDict, idfDict)
        except:
            print('Calculate error.')
            return False
        return tfidf

    def __cal_TF(self, wordDict, wash_token):
        tfDict = {}
        token_count = len(wash_token)
        for word, count in wordDict.items():
            tfDict[word] = count / token_count
        return tfDict

    def __cal_IDF(self, wordDict_lst):
        idfDict = dict.fromkeys(wordDict_lst[0], 0)
        N = len(wordDict_lst)
        for wordDict in wordDict_lst:
            for word, count in wordDict.items():
                if count > 0:
                    idfDict[word] += 1

        for word, ni in idfDict.items():
            idfDict[word] = math.log10((N+1) / (ni + 1))

        return idfDict

    def __cal_TFIDF(self, tf, idf):
        tfidf = {}
        for word, tf_val in tf.items():
            tfidf[word] = tf_val * idf[word]
        return tfidf

    def cal_simi(self, text1, text2, size='sen', method='euc'):
        try:
            token_1 = self.api_data.get_token(text1)
            token_2 = self.api_data.get_token(text2)
        except:
            print("Error with get token function") 
            return False
        
        try:
            if size == 'sen':
                vec_1 = self.api_data.get_vec(text1)['documentEmbedding']
                vec_2 = self.api_data.get_vec(text2)['documentEmbedding']
            elif size == 'word':
                vec_1 = self.api_data.get_vec(text1)['tokenEmbeddings']
                vec_2 = self.api_data.get_vec(text2)['tokenEmbeddings']
                #padding
                max_matrix_length = max(len(vec_1),len(vec_2))
                add = np.zeros(300)
                if max_matrix_length == len(vec_1):
                    vec_2.extend([list(add)]* (max_matrix_length-len(vec_2)))
                    vec_2 = np.array(vec_2).flatten()
                    vec_1 = np.array(vec_1).flatten()
                else:
                    vec_1.extend([list(add)]* (max_matrix_length-len(vec_1)))
                    vec_1 = np.array(vec_1).flatten()
                    vec_2 = np.array(vec_2).flatten()
            else:
                print('There is no size '.format(size))
                return False
        except:
            print("Error with get vector function") 
            return False
            


        if method == 'euc':
            score = self.__simi_cal_euc(vec_1, vec_2, size)
        elif method == 'cos':
            score = self.__simi_cal_cos(vec_1, vec_2, size)
        elif method == 'jac':
            score = self.__simi_cal_jac(token_1, token_2, size)
        else:
            print('There is no method '.format(method))
            return False
        
        return score

    def __simi_cal_euc(self, vec1,vec2,size):
        if size=='word':
            pass
        else:
            vec1=np.array(vec1)
            vec2=np.array(vec2)
        return float(np.sqrt(np.sum(np.square(vec1-vec2))))
    
    def __simi_cal_cos(self, vec1,vec2,size):
        if size=='word':
            pass
        else:
            vec1=np.array(vec1)
            vec2=np.array(vec2)
        return float(np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))

    def __simi_cal_jac(self, token_1, token_2, size):
        inter = len(list(set(token_1).intersection(token_2)))
        union = (len(token_1) + len(token_1)) - inter
        return float(inter) / union

    def cal_textRank(self, text, candidate_pos=['NOUN', 'PROPN', 'VERB'], top_k = 10, window_size=3, stop_word='default'):
        #damping coefficient
        damp = 0.85
        #convergence threshold
        min_conv = 1e-5
        iters = 10
        #keywords + weight
        key_weight = {}
        
        text = text.replace('\n', ' ')
        if len(text.replace(' ', '')) > 5000:
            print("Please control the length of input text")
            return False
            
            
        try:
            sentences = self.api_data.get_senTag(text)
        except:
            print("Error with get sentence tagging function")
            return False
    
        try:
            #filter pos tag
            pos_tag_lst = []
            for sen in sentences:
                pos_tag_lst.append(self.api_data.get_posTag(sen))
        except:
            print("Error with get pos tagging function")
            return False
    
        sen_filter_pos = []
        for pos_dic in pos_tag_lst:
            temp = []
            tokens = pos_dic['tokens']
            tags = pos_dic['posTags']
            for i in range(0,len(tags)):
                if tags[i] in candidate_pos:
                    temp.append(tokens[i])
            sen_filter_pos.append(temp)
    

        #determine language
        if stop_word == 'default':
            try:
                lang = self.api_data.get_lang(text)
            except:
                print("Error with get language function")
                return False
            if lang == 'zho':
                stop_word = 'pacakge_data/cn_stopwords.txt'
            elif lang == 'eng':
                stop_word = 'pacakge_data/en_stopwords.txt'
            else:
                stop_word = 'pacakge_data/other_stopwords.txt'
        elif stop_word == 'None':
            stop_word = 'pacakge_data/emp_stopwords.txt'

        try:
            stop_file = open(stop_word, 'r', encoding='utf-8')
            stopwords = stop_file.read().split("\n")
            stop_file.close()
        except:
            print('{} is not exist, please check the file!'.format(stop_word))
            return False

        washed_token = []
        for sen in sen_filter_pos:
            temp = []
            for char in sen:
                if char in stopwords:
                    pass
                else:
                    temp.append(char)
            washed_token.append(temp)
        
        vocab_dic = OrderedDict()
        count = 0
        for sen in washed_token:
            for token in sen:
                if token not in vocab_dic:
                    vocab_dic[token] = count
                    count += 1
                
        token_pair_lst = []
        for sen in washed_token:
            for i,word in enumerate(sen):
                for j in range(i+1,i+window_size):
                    if j>=len(sen):
                        break
                    if (word,sen[j]) not in token_pair_lst:
                        token_pair_lst.append((word,sen[j]))
                    
        #normalized matrix
        matrix_size = len(vocab_dic)
        matrix = np.zeros((matrix_size, matrix_size), dtype='float')
        for word_pair in token_pair_lst:
            word1 = vocab_dic[word_pair[0]]
            word2 = vocab_dic[word_pair[1]]
            matrix[word1][word2]=1
        
        #Symmeric matrix
        matrix = matrix +matrix.T-np.diag(matrix.diagonal())
    
        #normalize
        normalize = np.sum(matrix,axis=0)
        matrix_norm = np.divide(matrix,normalize,where=normalize!=0)
    
        #inital weight
        weight = np.array([1]*len(vocab_dic))
        pre_weight = 0
        for epoch in range(0,iters):
            weight = (1-damp)+damp*np.dot(matrix_norm,weight)
            if abs(pre_weight - sum(weight))  < min_conv:
                break
            else:
                pre_weight=sum(weight)
    
        for word,index in vocab_dic.items():
            key_weight[word]=weight[index]
        
        key_weight = OrderedDict(sorted(key_weight.items(), key=lambda t: t[1], reverse=True))
        count = 0
        keywords_lst = []
        for key in key_weight:
            keywords_lst.append((key,key_weight[key]))
            count += 1
            if count >= top_k:
                break
        return keywords_lst

    def relation_viz(self, text):
        text_lst = self.__check_limit(text)
        relations = []
        try:
            for t in text_lst:
                relations.extend(self.api_data.get_relation(t))
        except:
            print("Error with get relationship function")
            return False

        g = Graph('graph', filename='G', engine='neato')
        for relation_dic in relations:
            rela = relation_dic['predicate']
            arg1 = relation_dic['arg1']
            arg2 = relation_dic['arg2']
            g.node(arg1)
            g.node(arg2)
            g.edge(arg1,arg2, label=rela,len='1.50')
        
        g.view()
        return True

    def cal_classification(self, text, top_k = 3):
        try:
            classify = self.api_data.get_classification(text)
        except:
            print("Error with get classification function")
            return False
            
        if top_k > len(classify):
            print('The k exceed the number of categories')
            return False
        else:
            return classify[:top_k]

In [235]:
nlp=NLPbrl()

In [174]:
text="""simple test of calculate frequency."""

In [72]:
nlp.cal_frequency(text)

[('simple', 1), ('test', 1), ('calculate', 1), ('frequency', 1)]

In [91]:
nlp.word_viz(text, '', 10)

In [175]:
nlp.cal_textRank('simple test of calculate frequency.', candidate_pos=['NOUN', 'PROPN', 'VERB'], top_k = 10, window_size=3, stop_word='default')

[('test', 1.0), ('calculate', 1.0), ('frequency', 1.0)]

In [236]:
text="The goal of the package is to wrap into a set of R/Python functions a web REST API and offer a package for others to use those functions. The functions offered by the package should also take care of the minimum wrangling necessary to output the data in a viable format (that is, not as a raw binary file, unless a raw binary file is the most appropriate data format). You choose the API and the most appropriate input / output design of your functions. Think carefully of what part of the wrangling is of general interest (i.e., most or all of the users will want to perform it, and thus should be done in the package) and what part is only relevant as an example for the vignette (i.e., is too specific to be of general interest, and thus may just end up in the vignette.)"*20

In [83]:
nlp.cal_frequency(text)[0][1]

80

In [100]:
nlp.word_viz(text, 'test123/', 10)

Error with saving word cloud.


False

In [238]:
nlp.cal_textRank(text, candidate_pos=['NOUN', 'VERB'])

Please control the length of input text


False

In [179]:
text="""这是一个简单的代码测试代码块，用来测试中文的词频计算"""

In [56]:
nlp.cal_frequency(text)

[('代码', 2),
 ('测试', 2),
 ('一个', 1),
 ('简单', 1),
 ('块', 1),
 ('中文', 1),
 ('词频', 1),
 ('计算', 1)]

In [102]:
nlp.word_viz(text, '', 10)

True

In [180]:
nlp.cal_textRank(text, candidate_pos=['NOUN', 'VERB'])

[('测试', 1.586140773809524),
 ('代码', 1.152276488095238),
 ('中文', 1.0696949404761904),
 ('块', 1.053261607142857),
 ('词频', 0.9007574404761904),
 ('计算', 0.6608854166666667),
 ('一个', 0.5769833333333333)]

In [181]:
text=""

In [70]:
nlp.cal_frequency(text)

Error with get language function


False

In [104]:
nlp.word_viz(text, '', 10)

Error with get language function


False

In [182]:
nlp.cal_textRank(text)

Error with get sentence tagging function


False

In [136]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
    "let's begin our test.",
]
text = 'This is the test.'

In [137]:
nlp.key_extra_tfidf(text, corpus)['test']

0.23856062735983122

In [138]:
corpus = [
    "The cat sat on my bed",
    "The dog sat on my knees"
]
text = "The dog sat on my knees"

In [139]:
nlp.key_extra_tfidf(text, corpus,'None')['dog']

0.029348543175946873

In [145]:
nlp.key_extra_tfidf('', corpus,'None')

Error with input type and content.


False

In [146]:
nlp.key_extra_tfidf('', '','None')

Error with input type and content.


False

In [150]:
nlp.cal_simi('I am eating an apple', 'She is eating an apple', 'sen', 'euc')

0.40277734306421165

In [151]:
nlp.cal_simi('I am eating an apple', 'She is eating an apple', 'word', 'euc')

1.4605218363346508

In [152]:
nlp.cal_simi('I am eating an apple', 'She is eating an apple', 'sen', 'cos')

0.9188851939644076

In [153]:
nlp.cal_simi('I am eating an apple', 'She is eating an apple', 'word', 'cos')

0.7866876207707693

In [155]:
nlp.cal_simi('I am eating an apple', 'She is eating an apple',method = 'jac')

0.42857142857142855

In [169]:
nlp.cal_simi('', 'She is eating an apple', 'sen', 'euc')

Error with get token function


False

In [164]:
nlp.cal_simi('I am eating an apple', 'She is eating an apple', 'a')

There is no size 


False

In [165]:
nlp.cal_simi('I am eating an apple', 'She is eating an apple', 'sen','a')

There is no method 


False

In [183]:
rela_text = "FLIR Systems is headquartered in Oregon and produces thermal imaging, night vision, and infrared cameras and sensor systems.  According to the SEC’s order instituting a settled administrative proceeding, FLIR entered into a multi-million dollar contract to provide thermal binoculars to the Saudi government in November 2008.  Timms and Ramahi were the primary sales employees responsible for the contract, and also were involved in negotiations to sell FLIR’s security cameras to the same government officials.  At the time, Timms was the head of FLIR’s Middle East office in Dubai."

In [184]:
nlp.relation_viz(rela_text)

True

In [185]:
nlp.relation_viz('')

Error with get relationship function


False

In [187]:
cate_text = "Sony Pictures is planning to shoot a good portion of the new \"Ghostbusters\" in Boston as well."

In [188]:
nlp.cal_classification(cate_text)

[{'label': 'ARTS_AND_ENTERTAINMENT',
  'confidence': 0.06416648,
  'score': -0.01447566},
 {'label': 'SPORTS', 'confidence': 0.05782175, 'score': -0.11859164},
 {'label': 'TRAVEL', 'confidence': 0.05627946, 'score': -0.14562697}]

In [189]:
nlp.cal_classification(cate_text,100)

The k exceed the number of categories


False

In [190]:
nlp.cal_classification('')

Error with get classification function


False

## Unittest

In [6]:
import unittest
##########import NLPbrl


class TestProj(unittest.TestCase):
    @classmethod
    def setUpClass(self):
        self.nlp = NLPbrl()
        self.short_text_en = "simple test of calculate frequency."
        self.freq_short_res_en = [('simple', 1), ('test', 1), ('calculate', 1), ('frequency', 1)]
        self.long_text_en = "The goal of the package is to wrap into a set of R/Python functions a web REST API and offer a package for others to use those functions. The functions offered by the package should also take care of the minimum wrangling necessary to output the data in a viable format (that is, not as a raw binary file, unless a raw binary file is the most appropriate data format). You choose the API and the most appropriate input / output design of your functions. Think carefully of what part of the wrangling is of general interest (i.e., most or all of the users will want to perform it, and thus should be done in the package) and what part is only relevant as an example for the vignette (i.e., is too specific to be of general interest, and thus may just end up in the vignette.)"*20
        self.short_text_cn = "这是一个简单的代码测试代码块，用来测试中文的词频计算"
        self.freq_short_res_cn = [('代码', 2), ('测试', 2), ('一个', 1), ('简单', 1), ('块', 1), ('中文', 1), ('词频', 1), ('计算', 1)]
        self.empty_text = ""
        self.tf_idf_corpus1 = ['This is the first document.',
                        'This document is the second document.',
                        'And this is the third one.',
                        'Is this the first document?',
                        "let's begin our test.",]
        self.tf_idf_text1 = 'This is the test.'
        self.tf_idf_corpus2 = ["The cat sat on my bed",
                                "The dog sat on my knees"]
        self.tf_idf_text2 = "The dog sat on my knees"
        self.simi_text1 = 'I am eating an apple'
        self.simi_text2 = 'She is eating an apple'
        self.rank_short_res_en = [('test', 1.0), ('calculate', 1.0), ('frequency', 1.0)]
        self.rela_text = "FLIR Systems is headquartered in Oregon and produces thermal imaging, night vision, and infrared cameras and sensor systems.  According to the SEC’s order instituting a settled administrative proceeding, FLIR entered into a multi-million dollar contract to provide thermal binoculars to the Saudi government in November 2008.  Timms and Ramahi were the primary sales employees responsible for the contract, and also were involved in negotiations to sell FLIR’s security cameras to the same government officials.  At the time, Timms was the head of FLIR’s Middle East office in Dubai."
        self.cate_text = "Sony Pictures is planning to shoot a good portion of the new \"Ghostbusters\" in Boston as well."
        self.cate_res = [{'label': 'ARTS_AND_ENTERTAINMENT','confidence': 0.06416648,'score': -0.01447566},
                        {'label': 'SPORTS', 'confidence': 0.05782175, 'score': -0.11859164},
                        {'label': 'TRAVEL', 'confidence': 0.05627946, 'score': -0.14562697}]
        
    @classmethod
    def tearDownClass(self):
        print('successful')
        
    def test_cal_frequency(self):
        self.assertEqual(self.nlp.cal_frequency(self.short_text_en),self.freq_short_res_en)
        self.assertEqual(self.nlp.cal_frequency(self.long_text_en)[0][1],self.long_text_en.count('package'))
        self.assertEqual(self.nlp.cal_frequency(self.short_text_cn),self.freq_short_res_cn)
        self.assertEqual(self.nlp.cal_frequency(self.empty_text),False)

    def test_word_viz(self):
        self.assertEqual(self.nlp.word_viz(self.short_text_en, '', 10),True)
        self.assertEqual(self.nlp.word_viz(self.long_text_en, 'test123/', 10),False)
        self.assertEqual(self.nlp.word_viz(self.short_text_cn, '', 10),True)
        self.assertEqual(self.nlp.word_viz(self.empty_text, '', 10),False)

    def test_extra_tfidf(self):
        self.assertAlmostEqual(self.nlp.key_extra_tfidf(self.tf_idf_text1, self.tf_idf_corpus1)['test'],0.2386,2)
        self.assertAlmostEqual(self.nlp.key_extra_tfidf(self.tf_idf_text2, self.tf_idf_corpus2, 'None')['dog'],0.029,2)
        self.assertEqual(self.nlp.key_extra_tfidf(self.empty_text, self.tf_idf_corpus2),False)
        self.assertEqual(self.nlp.key_extra_tfidf(self.empty_text, self.empty_text),False)

    def test_cal_simi(self):
        self.assertAlmostEqual(self.nlp.cal_simi(self.simi_text1,self.simi_text2,'sen','euc'),0.40277734306421165)
        self.assertAlmostEqual(self.nlp.cal_simi(self.simi_text1,self.simi_text2,'word','euc'),1.46,2)
        self.assertAlmostEqual(self.nlp.cal_simi(self.simi_text1,self.simi_text2,'sen','cos'),0.92,2)
        self.assertAlmostEqual(self.nlp.cal_simi(self.simi_text1,self.simi_text2,'word','cos'),0.787,2)
        self.assertAlmostEqual(self.nlp.cal_simi(self.simi_text1,self.simi_text2,method='jac'),0.429,2)
        self.assertEqual(self.nlp.cal_simi(self.empty_text,self.simi_text2,'sen','euc'),False)
        self.assertEqual(self.nlp.cal_simi(self.simi_text1,self.simi_text2,'a'),False)
        self.assertEqual(self.nlp.cal_simi(self.simi_text1,self.simi_text2,'sen','a'),False)
        
    def test_textRank(self):
        self.assertEqual(self.nlp.cal_textRank(self.short_text_en),self.rank_short_res_en)
        self.assertAlmostEqual(self.nlp.cal_textRank(self.long_text_en, candidate_pos=['NOUN', 'VERB']),False)
        self.assertAlmostEqual(self.nlp.cal_textRank(self.short_text_cn, candidate_pos=['NOUN', 'VERB'])[0][1],1.586,2)
        self.assertEqual(self.nlp.cal_textRank(self.empty_text),False)

    def test_relation_viz(self):
        self.assertEqual(self.nlp.relation_viz(self.rela_text),True)
        self.assertEqual(self.nlp.relation_viz(self.empty_text),False)

    def test_cal_classification(self):
        self.assertEqual(self.nlp.cal_classification(self.cate_text),self.cate_res)
        self.assertEqual(self.nlp.cal_classification(self.cate_text, 100),False)
        self.assertEqual(self.nlp.cal_classification(self.empty_text),False)

In [7]:
test_cases = (TestProj)

def test_suite():
    suite = unittest.TestSuite()
    result = unittest.TestResult()
    suite.addTest(unittest.makeSuite(TestProj))
    runner = unittest.TextTestRunner()
    print(runner.run(suite))
test_suite()

The k exceed the number of categories


.

Error with get classification function


.

Error with get language function
Error with get token function
There is no size 


...

.

Error with get sentence tagging function
Error with saving word cloud.


.

Error with get language function
successful
<unittest.runner.TextTestResult run=7 errors=0 failures=0>



----------------------------------------------------------------------
Ran 7 tests in 31.279s

OK


In [214]:
a=TestProj()

In [216]:
a.nlp.key_extra_tfidf(a.tf_idf_text1, a.tf_idf_corpus1)

Error with get language function


False

In [217]:
a.tf_idf_text1

'This is the test.'

In [218]:
a.tf_idf_corpus1

['This is the first document.',
 'This document is the second document.',
 'And this is the third one.',
 'Is this the first document?',
 "let's begin our test."]